In [ ]:
import csv
import os
import math
import random
import time
import pandas as pd
import numpy as np
import openseespy.opensees as ops
import openseespy.postprocessing.ops_vis as opsv
from scipy.stats import zscore
from sklearn.model_selection import train_test_split
import torch
from torch import nn, optim
import torch.nn.functional as F
import math
from torch.optim.lr_scheduler import StepLR
import matplotlib.pylab as plt
import cv2
import time
start_time = time.time()

### Splitting to test, validation and train 80%/10%/10%
train = pd.read_hdf('train_norm.h5')
validation = pd.read_hdf('val_norm.h5')
test = pd.read_hdf('test_norm.h5')

### Splitting to input X and target y sets
### Converting to torch Tensor format (for torch trainig)
X_train = train[['P', 'My', 'Mz', 'fc', 'h']]
y_train = train[[ 'Width', 'Depth','As_total']]#, 'D_rebar']]

X_val = validation[['P', 'My', 'Mz', 'fc', 'h']]
y_val = validation[[ 'Width', 'Depth','As_total']]#, 'D_rebar']]

X_test = test[['P',  'My', 'Mz', 'fc', 'h']]
y_test = test[['Width', 'Depth','As_total']]#, 'D_rebar']]

### Converting to torch Tensor format (for torch trainig)

X_train = torch.from_numpy(X_train.to_numpy()).float()
y_train = torch.from_numpy(y_train.to_numpy()).float()

X_val = torch.from_numpy(X_val.to_numpy()).float()
y_val = torch.from_numpy(y_val.to_numpy()).float()


X_test = torch.from_numpy(X_test.to_numpy()).float()
y_test = torch.from_numpy(y_test.to_numpy()).float()

print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape, y_test.shape)

train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
val_dataset = torch.utils.data.TensorDataset(X_val, y_val)
test_dataset = torch.utils.data.TensorDataset(X_test, y_test)

batch_size = 128
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset,      # torch TensorDataset format
    batch_size=batch_size,  
    shuffle = True
)

val_loader = torch.utils.data.DataLoader(
    dataset=val_dataset,      # torch TensorDataset format
    batch_size=batch_size,      # mini batch size
)


#LeakyReLu    
class Net(nn.Module):
    def __init__(self,n_in, n_out, neurons):
        super(Net, self).__init__()
        self.layers = nn.Sequential(
          nn.Linear(n_in, 128),
          #nn.BatchNorm1d(neurons),
          nn.ReLU(),
          nn.Linear(128, neurons),
          #nn.BatchNorm1d(neurons),
          nn.ReLU(),
          nn.Linear(neurons, neurons),
          nn.ReLU(),
          nn.Linear(neurons, neurons),
          #nn.BatchNorm1d(neurons),
          nn.ReLU(),
          nn.Linear(neurons, neurons),
          #nn.BatchNorm1d(neurons),
          nn.ReLU(),
          nn.Linear(neurons, 512),
          #nn.BatchNorm1d(neurons),
          nn.ReLU(),
          nn.Linear(512, 512),
          #nn.BatchNorm1d(neurons),
          nn.ReLU(),
          nn.Linear(512, 512),
          #nn.BatchNorm1d(neurons),
          nn.ReLU(),
          nn.Linear(512, neurons),
          nn.ReLU(),
          nn.Linear(neurons, neurons),
          #nn.BatchNorm1d(neurons),
          nn.ReLU(),
          nn.Linear(neurons, neurons),
          #nn.BatchNorm1d(neurons),
          nn.ReLU(),
          nn.Linear(neurons, neurons),
          #nn.BatchNorm1d(neurons),
          nn.ReLU(),
          nn.Linear(neurons, 128),
          #nn.BatchNorm1d(neurons),
          nn.Dropout(p=0.1),
          nn.ReLU(),
          nn.Linear(128, n_out)
    )


    def forward(self, x):
        return self.layers(x)

net = Net(X_train.shape[1], y_train.shape[1], 256)
print(net)

#optimizer = optim.RMSprop(net.parameters(), lr=0.1, weight_decay=1e-4) 
learning_rate = 0.001
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
device = torch.device("cuda:8" if torch.cuda.is_available() else "cpu") #torch.device("cpu")
print(device)

net = net.to(device)
loss_func = nn.MSELoss()  
#loss_func = loss_func.to(device) 

# initialize a dictionary to store training history
H = {"train_loss": [], "val_loss": []}
epochs=150

#scheduler = StepLR(optimizer, step_size=5, gamma=0.5)
# calculate steps per epoch for training and validation set
trainSteps = len(train_loader.dataset) // batch_size
valSteps = len(val_loader.dataset) // batch_size
for epoch in range(epochs):
    
    # set the model in training mode
    net.train()
    # initialize the total training and validation loss
    totalTrainLoss = 0
    totalValLoss = 0
    total_loss = 0
    # loop over the training set
    for i,batch in enumerate(train_loader):
        #torch.cuda.empty_cache()
        x_train,y_target = batch
        
        # send the input to the device
        x_train, y_target = x_train.to(device),y_target.to(device)
        y_pred = net(x_train)
        
        train_loss = loss_func(y_pred, y_target)
    
        optimizer.zero_grad()   # clear gradients for next train
        train_loss.backward()   # backpropagation, compute gradients
        optimizer.step()        # apply gradients
        #scheduler.step()

        totalTrainLoss += train_loss 
    print("Epoch # %i, train_loss=%f"%(epoch, totalTrainLoss))

    with torch.no_grad():
        # set the model in evaluation mode
        net.eval()
        for i,batch in enumerate(val_loader):
            x_val,y_val = batch
            
            x_val, y_val = x_val.to(device),y_val.to(device)
            pred = net(x_val)
            totalValLoss += loss_func(pred, y_val)
    
    # calculate the average training and validation loss
    avgTrainLoss = totalTrainLoss / trainSteps
    avgValLoss = totalValLoss / valSteps

    # update our training history
    H["train_loss"].append(avgTrainLoss.cpu().detach().numpy())
    H["val_loss"].append(avgValLoss.cpu().detach().numpy())

### Saving the model
MODEL_PATH = 'fc_30.pth' 
torch.save(net, MODEL_PATH)

plt.style.use("ggplot")
plt.figure()
plt.plot(H["train_loss"], label="train_loss")
plt.plot(H["val_loss"], label="val_loss")
plt.title("Training and validation lossess")
plt.xlabel("Epoch #")
plt.ylabel("Loss")
plt.legend(loc="upper right")
plt.savefig(str(batch_size)+'_lr= '+str(learning_rate)+'_dp=0.3'+'_.png')

net.eval()

outp = net(X_test.to(device))

outp = outp.cpu()
out = outp.detach().numpy()
nn_out = pd.DataFrame(out, columns = ['Width', 'Depth', 'As_total'])

from sklearn.metrics import mean_squared_error

train_min = pd.read_hdf("train_min.h5")
train_min = train_min.drop(['P', 'My', 'Mz', 'fc', 'h'])

train_max = pd.read_hdf("train_max.h5")
train_max = train_max.drop(['P', 'My', 'Mz', 'fc', 'h'])

test_orig = pd.read_hdf("test.h5")
test_orig = test_orig.drop(columns=['P','My', 'Mz', 'fc', 'h'])


back_scaled_nn = train_min + nn_out*(train_max - train_min) 

error_scaled = mean_squared_error(back_scaled_nn, test_orig, multioutput='raw_values')

error = mean_squared_error(nn_out,y_test, multioutput='raw_values')
print(error_scaled)
print(error)

#[1.49517889e-03 1.35164139e-03 2.67062261e-06]
#[0.00233622 0.00211194 0.00361142]
## nn latest
#[1.70006513e-03 1.60931905e-03 2.43812537e-06]
#[0.00265635 0.00251456 0.00329702]

#[1.28692248e-03 1.26065387e-03 1.80810405e-06]
#[0.00201082 0.00196977 0.00244506]

a_file = open("Losses.csv", "w")


writer = csv.writer(a_file)
for key, value in H.items():
    writer.writerow([key, value])

a_file.close()

print("--- %s seconds ---" % (time.time() - start_time))






In [ ]:
train_loss = H['train_loss']
val_loss = H['val_loss']
df = pd.DataFrame(columns=['train loss', 'vall loss'])
df['train loss'] = train_loss
df['vall loss'] = val_loss
df.to_csv("loss_curve.csv")

In [ ]:

net.eval()

test_samples = pd.read_hdf("test_samples.h5")
X_test = torch.from_numpy(test_samples.to_numpy()).float()

outp = net(X_test.to(device))

outp = outp.cpu()
out = outp.detach().numpy()
nn_out = pd.DataFrame(out, columns = ['Width', 'Depth', 'As_total'])




from sklearn.metrics import mean_squared_error

train_min = pd.read_hdf("train_min.h5")
train_min = train_min.drop(['P', 'My', 'Mz', 'fc', 'h'])

train_max = pd.read_hdf("train_max.h5")
train_max = train_max.drop(['P', 'My', 'Mz', 'fc', 'h'])


back_scaled_nn = train_min + nn_out*(train_max - train_min)


back_scaled_nn.to_csv("nn_combo.csv")